In [1]:
# read the api key
f = open("sec_api.key", "r")
api_key = f.readline()

In [2]:
import pandas as pd
# this is for us to toy with an already downloaded csv
df_10k = pd.read_csv('./TSLA/csv/10-K.csv')
print(df_10k['linkToFilingDetails'])

0     https://www.sec.gov/Archives/edgar/data/131860...
1     https://www.sec.gov/Archives/edgar/data/131860...
2     https://www.sec.gov/Archives/edgar/data/131860...
3     https://www.sec.gov/ix?doc=/Archives/edgar/dat...
4     https://www.sec.gov/Archives/edgar/data/131860...
5     https://www.sec.gov/Archives/edgar/data/131860...
6     https://www.sec.gov/Archives/edgar/data/131860...
7     https://www.sec.gov/Archives/edgar/data/131860...
8     https://www.sec.gov/Archives/edgar/data/131860...
9     https://www.sec.gov/Archives/edgar/data/131860...
10    https://www.sec.gov/Archives/edgar/data/131860...
11    https://www.sec.gov/Archives/edgar/data/131860...
12    https://www.sec.gov/Archives/edgar/data/131860...
Name: linkToFilingDetails, dtype: object


In [6]:
urls = df_10k[['ticker', 
                'formType', 
                'periodOfReport',
                'filedAt', 
                'linkToFilingDetails']].rename(columns={'linkToFilingDetails': 'filingUrl'})

# display(urls)

urls['financialReportsUrl'] = urls['filingUrl'].apply(lambda url: '/'.join(url.split('/')[:-1]) + '/Financial_Report.xlsx')

urls.head(10)

,ticker,formType,periodOfReport,filedAt,filingUrl,financialReportsUrl
0,TSLA,10-K,2022-12-31,2023-01-30T21:29:15-05:00,https://www.sec.gov/Archives/edgar/data/131860...,https://www.sec.gov/Archives/edgar/data/131860...
1,TSLA,10-K,2021-12-31,2022-02-04T20:11:27-05:00,https://www.sec.gov/Archives/edgar/data/131860...,https://www.sec.gov/Archives/edgar/data/131860...
2,TSLA,10-K,2020-12-31,2021-02-08T07:27:23-05:00,https://www.sec.gov/Archives/edgar/data/131860...,https://www.sec.gov/Archives/edgar/data/131860...
3,TSLA,10-K,2019-12-31,2020-02-13T07:12:18-05:00,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/ix?doc=/Archives/edgar/dat...
4,TSLA,10-K,2018-12-31,2019-02-19T06:10:16-05:00,https://www.sec.gov/Archives/edgar/data/131860...,https://www.sec.gov/Archives/edgar/data/131860...
5,TSLA,10-K,2017-12-31,2018-02-23T06:07:43-05:00,https://www.sec.gov/Archives/edgar/data/131860...,https://www.sec.gov/Archives/edgar/data/131860...
6,TSLA,10-K,2016-12-31,2017-03-01T16:54:21-05:00,https://www.sec.gov/Archives/edgar/data/131860...,https://www.sec.gov/Archives/edgar/data/131860...
7,TSLA,10-K,2015-12-31,2016-02-24T16:17:56-05:00,https://www.sec.gov/Archives/edgar/data/131860...,https://www.sec.gov/Archives/edgar/data/131860...
8,TSLA,10-K,2014-12-31,2015-02-26T17:13:26-05:00,https://www.sec.gov/Archives/edgar/data/131860...,https://www.sec.gov/Archives/edgar/data/131860...
9,TSLA,10-K,2013-12-31,2014-02-26T16:02:51-05:00,https://www.sec.gov/Archives/edgar/data/131860...,https://www.sec.gov/Archives/edgar/data/131860...


In [ ]:
import requests
for single_filing in urls.iloc:
    print('selected filing: ', single_filing)
    financial_report_path = single_filing['financialReportsUrl'].replace('https://www.sec.gov/Archives/edgar/data/', '')

    base_url = 'https://archive.sec-api.io/' + financial_report_path

    render_api_url = base_url + '?token=' + api_key

    response = requests.get(render_api_url)

    file_name = f"{single_filing['ticker']}-{single_filing['periodOfReport']}-{single_filing['formType']}.xlsx"
    print(file_name)
    output = open('./TSLA/spreadsheets/' + file_name, 'wb')
    output.write(response.content)
    output.close()


In [7]:
# load every sheet into a dataframe
# also, %pip install openpyxl to load the excel sheet
all_sheets_sif = pd.read_excel('spreadsheets/SIF-2023-09-30-10-K.xlsx', sheet_name=None)
all_sheets_mrdb = pd.read_excel('spreadsheets/MRDB-2023-09-30-10-K.xlsx', sheet_name=None)
all_sheets_bdl = pd.read_excel('spreadsheets/BDL-2023-09-30-10-K.xlsx', sheet_name=None)
all_sheets_msbb = pd.read_excel('spreadsheets/MSBB-2023-09-30-10-K.xlsx', sheet_name=None)
all_sheets_wbba = pd.read_excel('spreadsheets/WBBA-2023-07-31-10-K.xlsx', sheet_name=None)

print('SIF', len(all_sheets_sif))
print('MRDB', len(all_sheets_mrdb))
print('BDL', len(all_sheets_bdl))
print('MSSB', len(all_sheets_msbb))
print('WBBA', len(all_sheets_wbba))

#for s in all_sheets.keys():
#    print(s)

sheets = pd.DataFrame({
    'SIF': all_sheets_sif,
    'BDL': all_sheets_bdl,
    'MRDB': all_sheets_mrdb,
    'MSSB': all_sheets_msbb,
    'WBBA': all_sheets_wbba
    })

display(sheets)

SIF 82
MRDB 82
BDL 71
MSSB 70
WBBA 34


,SIF,BDL,MRDB,MSSB,WBBA
Cover Page,Cover Page - ...,NaN,NaN,NaN,NaN
Audit Information,Audit Information 12 Months End...,NaN,Audit Information 12 Months...,NaN,NaN
Consolidated Statements of Oper,Consolidated Statements of Operations - USD...,NaN,Consolidated Statements of Operations and C...,NaN,NaN
Consolidated Statements of Comp,Consolidated Statements of Comprehensive Inc...,NaN,NaN,Consolidated Statements of Comprehensive Inc...,NaN
Consolidated Balance Sheets,Consolidated Balance Sheets - USD ($) $ in ...,NaN,Consolidated Balance Sheets - USD ($) $ in ...,Consolidated Balance Sheets - ...,NaN
...,...,...,...,...,...
Note 9 - Franchise Rights (Deta,NaN,NaN,NaN,NaN,Note 9 - Franchise Rights (Details Narrative...
Note 10 - Accrued Expenses an_2,NaN,NaN,NaN,NaN,Note 10 - Accrued Expenses and Other Payable...
Note 11 - Shareholder Equity (D,NaN,NaN,NaN,NaN,Note 11 - Shareholder Equity (Details Narrat...
Note 12 - General and Adminis_2,NaN,NaN,NaN,NaN,Note 12 - General and Administrative Expense...


In [44]:
# FYI - a loaded excel sheet is a python dict
# a single sheet is a DataFrame
# a single row is a series
income = all_sheets_bdl['CONSOLIDATED STATEMENTS OF INCO']
# display(income)

# print(income.iloc[0].keys()[0])
print(income.iloc[2].to_json())

display(income)


{"CONSOLIDATED STATEMENTS OF INCOME - USD ($) $ in Thousands":"Restaurant food sales","12 Months Ended":107238,"Unnamed: 2":97429}


,CONSOLIDATED STATEMENTS OF INCOME - USD ($) $ in Thousands,12 Months Ended,Unnamed: 2
0,NaN,"Sep. 30, 2023","Oct. 01, 2022"
1,Revenues:,,
2,Restaurant food sales,107238,97429
3,Restaurant bar sales,29000,26198
4,Package store sales,35187,31692
5,Franchise related revenues,1857,1826
6,Rental income,951,814
7,Other operating income,163,173
8,Total,174396,158132
9,Cost of merchandise sold:,,
